In [1]:
import scipy.io as sio
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [8]:
#load al preprocess all data
soundData = sio.loadmat('soundData_small.mat') #open data .mat file
soundData =  pd.DataFrame(soundData['sound_data_small'])#get only actual data and convert to dataframe
soundData = (soundData-soundData.mean())/soundData.std()#normalisation

soundData_HL = sio.loadmat('soundData_HL_small.mat') #open data .mat file
soundData_HL =  pd.DataFrame(soundData_HL['sound_data_small'])#get only actual data and convert to dataframe
soundData_HL = (soundData_HL-soundData_HL.mean())/soundData_HL.std()#normalisation

soundData_CI = sio.loadmat('soundData_CI_small.mat') #open data .mat file
soundData_CI =  pd.DataFrame(soundData_CI['sound_data_small'])#get only actual data and convert to dataframe
soundData_CI = (soundData_CI-soundData_CI.mean())/soundData_CI.std()#normalisation

soundData_afterCI = sio.loadmat('soundData_afterCI_small.mat') #open data .mat file
soundData_afterCI =  pd.DataFrame(soundData_afterCI['sound_data_small'])#get only actual data and convert to dataframe
soundData_afterCI = (soundData_afterCI-soundData_afterCI.mean())/soundData_afterCI.std()#normalisation

In [3]:
# labels = pd.DataFrame(sio.loadmat('Stim288.mat')['labels'][0])
labels = sio.loadmat('Stim288.mat')['labels'][0]
# labels = labels.transpose()
pd.DataFrame(labels)
soundLabels = pd.DataFrame(columns = ['label'])
for label in labels:
    soundLabels.loc[len(soundLabels.index)] = label#somehow the original array isnt good for svm, so i copy the data manually, idk why this does work
soundLabels = np.asarray(soundLabels).ravel() #flatten the array so numpy is happy
# print(np.asarray(labels))

In [17]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(soundData, soundLabels, test_size=0.25, shuffle = True, stratify=soundLabels)
print(pd.DataFrame(y_train, columns = ["label"]).groupby(["label"]).size())

#create svm classifier
clf = SVC(kernel='rbf')

#fit svm on training data
clf.fit(x_train, y_train)

predictions = clf.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print("Healthy class Accuracy (normalized): " + str(accuracy))

label
animal    36
music     36
nature    36
speech    36
tools     36
voice     36
dtype: int64
Healthy class Accuracy (normalized): 0.6111111111111112


In [18]:
#k-fold cross validation
k = 4
kf = KFold(n_splits=k, random_state=None, shuffle = True)
result = cross_val_score(clf , soundData, soundLabels, cv = kf)
 
print("Healthy class avg accuracy from CV: {}".format(result.mean()))

Healthy class avg accuracy from CV: 0.6458333333333334


In [19]:
#accuracy on other scenarios
#create svm classifier with all healthy data instead of train set
clf2 = clf
#fit svm on training data
clf2.fit(soundData, soundLabels)

predictions_HL = clf2.predict(soundData_HL)
accuracy_HL = accuracy_score(soundLabels, predictions_HL)
print("Hearing loss class Accuracy: " + str(accuracy_HL))

predictions_CI = clf2.predict(soundData_CI)
accuracy_CI = accuracy_score(soundLabels, predictions_CI)
print("Cochlear implant class Accuracy: " + str(accuracy_CI))

predictions_afterCI = clf2.predict(soundData_afterCI)
accuracy_afterCI = accuracy_score(soundLabels, predictions_afterCI)
print("Cochlear implant after Hearing loss class Accuracy: " + str(accuracy_afterCI))

Hearing loss class Accuracy: 0.5416666666666666
Cochlear implant class Accuracy: 0.4861111111111111
Cochlear implant after Hearing loss class Accuracy: 0.8090277777777778
